#### **Project: Moving Average Crossover Trading Strategy (with list input)**

**Objective:** Create a Python script that simulates a simple trading strategy using historical stock prices. The strategy is based on the crossing of short-term and long-term moving averages.

---

#### **Introduction to the Strategy**
This script simulates a common **Moving Average Crossover** trading strategy.
* A **short moving average (MA)** is a simple average of prices over a shorter period (e.g., 5 days).
* A **long moving average (MA)** is an average over a longer period (e.g., 10 or 20 days).
* **Buy Signal**: Generated when the short MA crosses *above* the long MA. This suggests upward momentum.
* **Sell Signal**: Generated when the short MA crosses *below* the long MA. This suggests downward momentum.

#### **Import Libraries and Define Sample Data**


In [1]:
import matplotlib.pyplot as plt

#  Sample Historical Stock Data (date, price)
data = [
    ("2021-01-01", 100),
    ("2021-01-02", 102),
    ("2021-01-03", 104),
    ("2021-01-04", 101),
    ("2021-01-05", 105),
    ("2021-01-06", 107),
    ("2021-01-07", 110),
    ("2021-01-08", 108),
    ("2021-01-09", 106),
    ("2021-01-10", 104),
    ("2021-01-11", 103),
    ("2021-01-12", 102),
    ("2021-01-13", 101),
    ("2021-01-14", 105),
    ("2021-01-15", 108),
    ("2021-01-16", 110),
    ("2021-01-17", 112),
    ("2021-01-18", 115),
    ("2021-01-19", 117),
    ("2021-01-20", 120),
]

# Extract just the prices for easier calculations
prices = [item[1] for item in data]
dates = [item[0] for item in data]

print("Historical data loaded.")
print(f"Total data points: {len(prices)} days")

Historical data loaded.
Total data points: 20 days


#### **Processing Data: Calculate Moving Averages**
This section defines the periods for our short and long moving averages and then calculates them using loops. Since our dataset is small, the long MA period is set to 10 days.

In [3]:
# Define moving average periods
short_ma_period = 5
long_ma_period = 10 # Adjusted to 10 for this small dataset size

# Initialize lists to store moving averages, filling with zeros initially
short_m_avg = [0.0] * len(prices)
long_m_avg = [0.0] * len(prices)

# Calculate moving averages using loops
for i in range(len(prices)):
    # Calculate Short MA
    if i >= short_ma_period - 1:
        short_m_avg[i] = sum(prices[i - short_ma_period + 1 : i + 1]) / short_ma_period

    # Calculate Long MA
    if i >= long_ma_period - 1:
        long_m_avg[i] = sum(prices[i - long_ma_period + 1 : i + 1]) / long_ma_period

print("--- Moving Averages Calculated ---")
print(f"Short MA Period: {short_ma_period} days")
print(f"Long MA Period: {long_ma_period} days")
print(f"First valid Short MA (Day {short_ma_period}): {short_m_avg[short_ma_period-1]:.2f}")
print(f"First valid Long MA (Day {long_ma_period}): {long_m_avg[long_ma_period-1]:.2f}")
print("-" * 60)

--- Moving Averages Calculated ---
Short MA Period: 5 days
Long MA Period: 10 days
First valid Short MA (Day 5): 102.40
First valid Long MA (Day 10): 104.70
------------------------------------------------------------


#### **Generating Signals and Simulating Trades**
This is where the trading strategy comes to life. We iterate through the data, generate buy/sell signals based on MA crossovers, and simulate trades, updating our cash and shares. A `trade_log` keeps track of all actions.

In [6]:
# Initialize trading variables within this cell for proper execution
initial_cash = 1000.0 # Redefine or ensure this is from a previous cell
current_cash = initial_cash
shares = 0
portfolio_value = initial_cash
in_position = False # Track if we currently hold shares

trade_log = [] # To store details of each trade

print("\n--- Simulating Trades ---")
print(f"Starting Portfolio Value: ${initial_cash:.2f}")

for i in range(len(prices)):
    # Only generate signals once both MAs have valid values
    if short_m_avg[i] != 0 and long_m_avg[i] != 0:
        current_price = prices[i]

        # Buy signal: Short MA crosses above Long MA AND we are not already holding shares
        if short_m_avg[i] > long_m_avg[i] and not in_position:
            # Confirm a true crossover (short MA was below/equal previous day)
            if i > 0 and short_m_avg[i-1] <= long_m_avg[i-1]:
                buy_quantity = int(current_cash / current_price) # Buy as many shares as possible
                if buy_quantity > 0:
                    shares += buy_quantity
                    current_cash -= (buy_quantity * current_price)
                    portfolio_value = current_cash + (shares * current_price)
                    in_position = True
                    trade_log.append(f"{dates[i]}: BUY {buy_quantity} shares @ ${current_price:.2f} | Cash: ${current_cash:.2f}, Shares: {shares}, Portfolio: ${portfolio_value:.2f}")

        # Sell signal: Short MA crosses below Long MA AND we are currently holding shares
        elif short_m_avg[i] < long_m_avg[i] and in_position:
            # Confirm a true crossover (short MA was above/equal previous day)
            if i > 0 and short_m_avg[i-1] >= long_m_avg[i-1]:
                if shares > 0: # Ensure we actually have shares to sell
                    current_cash += (shares * current_price) # Sell all shares
                    portfolio_value = current_cash # Shares will become 0 after sale
                    trade_log.append(f"{dates[i]}: SELL {shares} shares @ ${current_price:.2f} | Cash: ${current_cash:.2f}, Shares: 0, Portfolio: ${portfolio_value:.2f}")
                    shares = 0 # Update shares to 0
                    in_position = False

    # Update portfolio value for current day even if no trade occurs
    if i >= long_ma_period -1: # Only track value once MAs are valid
        portfolio_value = current_cash + (shares * prices[i])

print("\n--- Trade Log ---")
if not trade_log:
    print("No trades were executed based on the strategy.")
else:
    for log_entry in trade_log:
        print(log_entry)
print("-" * 60)


--- Simulating Trades ---
Starting Portfolio Value: $1000.00

--- Trade Log ---
2021-01-17: BUY 8 shares @ $112.00 | Cash: $104.00, Shares: 8, Portfolio: $1000.00
------------------------------------------------------------


#### **Printing Final Results**
This cell calculates and outputs the final portfolio value, comparing it to the initial capital to show the net gain or loss from the simulation.

In [7]:
# Calculate final portfolio value by valuing any outstanding shares at the last known price
final_portfolio_value = current_cash + (shares * prices[-1])

print("\n--- Final Simulation Results ---")
print(f"Starting Portfolio Value: ${initial_cash:.2f}")
print(f"Final Cash: ${current_cash:.2f}")
print(f"Final Shares Held: {shares}")
print(f"Final Portfolio Value: ${final_portfolio_value:.2f}")

gain_loss = final_portfolio_value - initial_cash
print(f"Net Gain/Loss: ${gain_loss:.2f}")

if gain_loss > 0:
    print("Strategy Result: PROFIT! Well done.")
elif gain_loss < 0:
    print("Strategy Result: LOSS. Consider adjusting parameters or strategy.")
else:
    print("Strategy Result: BROKE EVEN.")

print("-" * 60)
print("Simulation complete.")


--- Final Simulation Results ---
Starting Portfolio Value: $1000.00
Final Cash: $104.00
Final Shares Held: 8
Final Portfolio Value: $1064.00
Net Gain/Loss: $64.00
Strategy Result: PROFIT! Well done.
------------------------------------------------------------
Simulation complete.
